In [ ]:
import static_sox
import os
static_sox.add_paths()  # Only install if sox is not available.
os.system("sox --help")

In [ ]:
%load_ext autoreload
import os
import sys
rubberband_path = 'C:\\Users\\ryanw\\Documents\\Code\\Modules\\NX\\rubberband'
if rubberband_path not in sys.path:
    print('Adding rubberband folder to sys.path.')
    sys.path.append(rubberband_path)

if rubberband_path not in os.environ['PATH']:
    print('Adding rubberband folder to PATH.')
    os.environ['PATH'] += f';{rubberband_path}'

if 'rubberband' not in os.environ:
    print('Adding rubberband.exe as environment variable.')
    os.environ['rubberband'] = f'{rubberband_path}\\rubberband.exe'

In [ ]:
%autoreload

from xpaudio import *
from wavegraph import *
from stftpitchshift import StftPitchShift, pitcher
import sox
from IPython.display import Audio, display, clear_output, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from typing import TypeVar
from collections import deque
from pydantic import ValidationError
#import IPython.display
import numpy as np
import array
from IPython.display import clear_output
from copy import deepcopy, copy
import time
import pandas as pd
import itertools
from xperiment import *
from math import log, log2
import pyrubberband as pyrb
from pyrubberband.pyrb import __rubberband
from pedalboard import VST3Plugin

In [1]:
from msl.loadlib import LoadLibrary
from win32com import client
import os
import ctypes as ct
import rubberwrapper
from pydantic import create_model, BaseModel
import clr
from pydict2class import PyDict2Class
from pprint import pprint
from enum import Enum
# import mappingproxy
from typing import Mapping, Any, Dict, List, Tuple, Union, Optional, Sequence, Iterable, Callable, TypeVar, Generic, Set, Deque, Iterator, NamedTuple, FrozenSet
from itertools import chain
from pyclibrary import CLibrary
from collections import UserString
from IPython.core.oinspect import Inspector, object_info
from copy import deepcopy
import pandas as pd


In [3]:
clr.FindAssembly('IAsyncStateMachine')


In [10]:
NWaves_Loc = r'C:\Users\ryanw\Documents\Code\Modules\NX\nwaves.0.9.6\lib\net5.0\NWaves.dll'
NWaves2_Loc = r'C:\Users\ryanw\Documents\Code\Modules\NX\nwaves.0.9.6\lib\netstandard2.0\NWaves.dll'
NWavesNet = LoadLibrary(
    NWaves2_Loc, 
    'net'
)

In [9]:
rw_loc = os.path.dirname(rubberwrapper.__file__)
NWaves_Loc = r'C:\Users\ryanw\Documents\Code\Modules\NX\nwaves.0.9.6\lib\net5.0\NWaves.dll'

RubberbandNet = LoadLibrary(
    f'{rw_loc}/rubberband-sharp.dll', 
    'net', 
    mode=ct.RTLD_GLOBAL
)
RubberbandWin = LoadLibrary(
    f'{rw_loc}/rubberband-dll-x64.dll', 
    'windll', 
    mode=ct.RTLD_GLOBAL
)

#BinderWin = CLibrary(rw_loc + '/rubberband-dll-x64.dll')
#BinderNet = CLibrary(rw_loc + '/rubberband-sharp.dll')


In [20]:
from inspect import *
getmembers(NWavesNet.lib.NWaves.Transforms)


[('Base', <module 'NWaves.Transforms.Base'>),
 ('CepstralTransform', NWaves.Transforms.CepstralTransform),
 ('Dct1', NWaves.Transforms.Dct1),
 ('Dct2', NWaves.Transforms.Dct2),
 ('Dct3', NWaves.Transforms.Dct3),
 ('Dct4', NWaves.Transforms.Dct4),
 ('FastDct2', NWaves.Transforms.FastDct2),
 ('FastDct3', NWaves.Transforms.FastDct3),
 ('FastDct4', NWaves.Transforms.FastDct4),
 ('FastMdct', NWaves.Transforms.FastMdct),
 ('Fft', System.Fft),
 ('Fft64', System.Fft64),
 ('Goertzel', NWaves.Transforms.Goertzel),
 ('HartleyTransform', NWaves.Transforms.HartleyTransform),
 ('HilbertTransform', NWaves.Transforms.HilbertTransform),
 ('HilbertTransform64', NWaves.Transforms.HilbertTransform64),
 ('IDct', NWaves.Transforms.IDct),
 ('MagnitudePhaseList', System.MagnitudePhaseList),
 ('Mdct', NWaves.Transforms.Mdct),
 ('MellinTransform', NWaves.Transforms.MellinTransform),
 ('RealFft', System.RealFft),
 ('RealFft64', System.RealFft64),
 ('Stft', System.Stft),
 ('Wavelets', <module 'NWaves.Transforms.W

In [12]:
look.info(NWavesNet.lib.NWaves.Transforms, detail_level=1)

{'type_name': 'ModuleObject',
 'base_class': "<class 'CLR.ModuleObject'>",
 'string_form': "<module 'NWaves.Transforms'>",
 'namespace': None,
 'length': None,
 'file': 'c:\\users\\ryanw\\documents\\code\\modules\\nx\\unknown',
 'definition': None,
 'docstring': 'Namespace containing types from the following assemblies:\n\n- NWaves, Version=0.9.6.0, Culture=neutral, PublicKeyToken=null',
 'source': None,
 'init_definition': None,
 'class_docstring': None,
 'init_docstring': None,
 'call_def': None,
 'call_docstring': None,
 'ismagic': False,
 'isalias': False,
 'isclass': None,
 'found': True,
 'name': '',
 'subclasses': None}

In [3]:
class XString(UserString):

    @property
    def T(self):
        _tdata = deepcopy(self._tdata)
        data = deepcopy(self.data)
        self._tdata = data
        self.data = _tdata
        return self

    def iTab(self, n=1):
        self._tdata = self.data + '\t'*n
        self.data = '\t'*n + self.data
        return self
    
    def iLine(self, n=1):
        self._tdata = self.data + '\n'*n
        self.data = '\n'*n + self.data
        return self
    
    def iSpace(self, n=1):
        self._tdata = self.data + ' '*n
        self.data = ' '*n + self.data
        return self
    
    def iAllocate(self, n=10):
        self._Aloc = n 
        self._AlocData = []
        return self
    
    def iAlok(self, value):
        return self.iAllocate(value)
    
    def i2Alok(self, valueA, valueB):
        A = XString(valueA)
        B = XString(valueB)
        Alen = len(A)
        Blen = len(B)
        padding = self._Aloc - (Alen + Blen)
        Aloc = XString(A)
        Aloc.iSpace(padding).T
        Aloc += B
        self._AlocData.append(Aloc)
        return Aloc
    
    def iGetALok(self):
        return self._AlocData
    
    def iAlokDlim(self, delim='\n'):
        if delim == '_':
            delim = '\n' + ('_'*self._Aloc) + '\n\n' 
        
        return delim.join([str(x) for x in self._AlocData])


In [4]:
class CodeXDoc:
    def __init__(self, name=None):
        self.name = name
        self.arg_delim = ', '
        self.data = XString('')

    def __repr__(self):
        return self.data.data.__repr__()
    
    def STR(self):
        return self.data.data.__str__()
    
    def add_import(self, import_str):
        self.data += XString(f'import {import_str}').iLine().T

    def add_class(self, class_str):
        self.data += XString(f'class {class_str}:').iLine(2)

    def add_init(self, init_args):
        self.data += XString(f'def __init__({self.arg_delim.join(init_args)}):').iTab().iLine()

    def add_attribute(self, attr_name, attr_value=None):
        self.data += XString(f'self.{attr_name} = {attr_value}').iTab(2).iLine()

    def add_method(self, method_name, method_args, method_body):
        self.data.iLine().T
        self.data += XString(f'def {method_name}({self.arg_delim.join(method_args)}):').iTab().iLine()
        self.data += XString(f'{method_body}').iTab(2).iLine()

class XGenSettings(Settings):
    class Config:
        arbitrary_types_allowed = True
        extra = 'allow'
        allow_mutation = True

    name: str = None
    module: object
    add_init: bool = True
    class_type: object
    attr_type: object
    func_type: object

class XGenFunction(Settings):
    funtion_name: str
    call_signature: str
    body: str = None
    call: object = None
    return_type: object = None

class XGenAttributes(Settings):
    attribute_name: str
    attribute_value: object = None
    attribute_type: object = None

class XGenClassResources:
    class_name: str = None
    class_signature: str = None
    attributes: dict = {}
    methods: list = []

class XType:
    def __init__(self, comparison_func: Callable):
        self.comparison_func = comparison_func

    def __eq__(self, other):
        return self.comparison_func(other)

class CodeXGen:
    def __init__(self, name=None, obj=None):
        self._inspector = None
        self.settings = XGenSettings(name=name)
        self.generated_resources = XGenClassResources()
        if obj is not None: self.settings.module = obj


    @property
    def analyze(self):
        if self._inspector is None:
            self._inspector = Inspector()
        return self._inspector
    
    @property
    def _path(self):
        return f'{str(self.settings.module.__module__)}.{str(self.settings.module.__name__)}'
    
    def _data(self, obj=None):
        return obj.__dict__ if obj is not None else self.settings.module.__dict__
    
    
    def _info(self, obj=None):
        if obj is not None: return self.analyze.info(obj)
        return self.analyze.info(self.settings.module)
    
    def name_from_repr(self, obj):
        if obj is not None: return obj.__repr__().split("'")[1]
        return self.settings.module.__repr__().split("'")[1]

    
    def _call_signature(self, obj=None):
        if obj is not None: return self.analyze.call_signature(obj)
        return self.analyze.pdef(self.settings.module)

    def create_xtype(self, comparison_func: Callable):
        return XType(comparison_func)
    
    def _module(self, obj):
        return obj.__module__ if obj is not None else self.settings.module.__module__
    
    def _get_functions(self, obj=None):
        if obj is not None: funcs = {k:v for (k, v) in self._data(obj).items() if v == self.settings.func_type}
        else: funcs = {k:v for (k, v) in self._data().items() if v == self.settings.func_type}
        for k, v in funcs.items():
            call_sig = self._info(obj=v)['init_definition']
            name = k.lower()
            return_type = f"{self._path}.{k}{call_sig}"
            method_config = XGenFunction(funtion_name=name, call_signature=self._call_signature(v), return_type=return_type)
            self.generated_resources.methods.append(method_config)

    def _get_attributes(self, obj=None):
        if obj is not None: attrs = {k:v for (k, v) in self._data(obj).items() if v == self.settings.attr_type}
        else: attrs = {k:v for (k, v) in self._data().items() if v == self.settings.attr_type}
        for k, v in attrs.items():
            name = k
            if v is not None: value = f"{self._path}.{k}"
            attribute_config = XGenAttributes(attribute_name=name, attribute_value=None, attribute_type=None)
            self.generated_resources.attributes[name] = attribute_config
    
    def 

SyntaxError: invalid syntax (Temp/ipykernel_19960/457645869.py, line 128)

In [101]:

y = XString('')
y.iAllocate(20)

y.iAddAlloc('hello', 'world')
y.iAddAlloc('XString', 'Alloc')
y.iAddAlloc('Is', 'Sexy')
print(y.iAllocDelim('_'))

hello          world
____________________

XString        Alloc
____________________

Is              Sexy


In [112]:
tdoc = CodeXDoc()
tdoc.add_import('TestModule')
tdoc.add_class('TestClass')
tdoc.add_init(['self', 'test'])
tdoc.add_attribute('test', 'test')
tdoc.add_method('test_method', ['self', 'test'], 'print(test)')
print(tdoc.STR())

import TestModule


class TestClass:
	def __init__(self, test):
		self.test = test

	def test_method(self, test):
		print(test)


In [7]:

d = {k:v for (k, v) in RubberbandNet.lib.RubberBand.RubberBandStretcher.__dict__.items() if not k.startswith('__')}

In [8]:
def func_type(other):
    return 'CLR.MethodObject' in str(type(other))

def get_name_from_repr(obj):
    return obj.__repr__().split("'")[1]

options = d['Options']
reset = d['Reset']
FuncType = XType(func_type)
options == FuncType, reset == FuncType

(False, True)

In [77]:
RubberbandNet.lib.RubberBand.RubberBandStretcher.__name__
RubberbandNet.lib.RubberBand.RubberBandStretcher.DefaultOptions.__subclasshook__

AttributeError: 'Options' object has no attribute '__name__'

In [69]:
look.info(d['Reset'])

{'type_name': 'MethodObject',
 'base_class': "<class 'CLR.MethodObject'>",
 'string_form': "<method 'Reset'>",
 'namespace': None,
 'length': None,
 'file': 'c:\\users\\ryanw\\documents\\code\\modules\\nx\\unknown',
 'definition': None,
 'docstring': 'Void Reset()',
 'source': None,
 'init_definition': None,
 'class_docstring': None,
 'init_docstring': None,
 'call_def': None,
 'call_docstring': None,
 'ismagic': False,
 'isalias': False,
 'isclass': None,
 'found': True,
 'name': '',
 'subclasses': None}

In [11]:
look = Inspector()
RBS_Info = look.info(RubberbandNet.lib.RubberBand.RubberBandStretcher)

In [79]:
RBS_Info
linfo = look.info(RubberbandNet.lib.RubberBand.RubberBandStretcher)
#look.pinfo(RubberbandNet.lib.RubberBand.RubberBandStretcher, info=linfo)
type(linfo['init_definition'])

str

In [18]:

clr.AddReference('CLR.MethodObject')

FileNotFoundException: Could not load file or assembly 'CLR.MethodObject' or one of its dependencies. The system cannot find the file specified.
File name: 'CLR.MethodObject'
   at System.Reflection.RuntimeAssembly._nLoad(AssemblyName fileName, String codeBase, Evidence assemblySecurity, RuntimeAssembly locationHint, StackCrawlMark& stackMark, IntPtr pPrivHostBinder, Boolean throwOnFileNotFound, Boolean forIntrospection, Boolean suppressSecurityChecks)
   at System.Reflection.RuntimeAssembly.InternalLoadAssemblyName(AssemblyName assemblyRef, Evidence assemblySecurity, RuntimeAssembly reqAssembly, StackCrawlMark& stackMark, IntPtr pPrivHostBinder, Boolean throwOnFileNotFound, Boolean forIntrospection, Boolean suppressSecurityChecks)
   at System.Reflection.Assembly.Load(AssemblyName assemblyRef)
   at Python.Runtime.AssemblyManager.LoadAssembly(AssemblyName name)
   at Python.Runtime.CLRModule.AddReference(String name)

WRN: Assembly binding logging is turned OFF.
To enable ass

In [49]:
look = Inspector()
look.info(RubberbandNet.lib.RubberBand.RubberBandStretcher, detail_level=1)

{'type_name': 'CLRMetatype',
 'base_class': "<class 'clr._internal.CLRMetatype'>",
 'string_form': "<class 'RubberBand.RubberBandStretcher'>",
 'namespace': None,
 'length': None,
 'file': 'c:\\users\\ryanw\\documents\\code\\modules\\nx\\rubberwrapper\\rubberband-sharp.dll',
 'definition': None,
 'docstring': 'Void .ctor(Int32, Int32, Options, Double, Double)',
 'source': None,
 'init_definition': '(\n    sampleRate,\n    channels,\n    options=<RubberBand.Options object at 0x000001A518836F00>,\n    initialTimeRatio=1.0,\n    initialPitchScale=1.0,\n)',
 'class_docstring': None,
 'init_docstring': 'Void .ctor(Int32, Int32, Options, Double, Double)',
 'call_def': None,
 'call_docstring': None,
 'ismagic': False,
 'isalias': False,
 'isclass': True,
 'found': True,
 'name': '',
 'subclasses': ''}

In [54]:
object_info(obj = RubberbandNet.lib.RubberBand.RubberBandStretcher.__dict__)

{'type_name': None,
 'base_class': None,
 'string_form': None,
 'namespace': None,
 'length': None,
 'file': None,
 'definition': None,
 'docstring': None,
 'source': None,
 'init_definition': None,
 'class_docstring': None,
 'init_docstring': None,
 'call_def': None,
 'call_docstring': None,
 'ismagic': None,
 'isalias': None,
 'isclass': None,
 'found': None,
 'name': None,
 'obj': mappingproxy({'DefaultOptions': <field 'Options DefaultOptions'>,
               'PercussiveOptions': <field 'Options PercussiveOptions'>,
               'Options': RubberBand.Options,
               'Dispose': <method 'Dispose'>,
               'Reset': <method 'Reset'>,
               'SetTimeRatio': <method 'SetTimeRatio'>,
               'SetPitchScale': <method 'SetPitchScale'>,
               'GetTimeRatio': <method 'GetTimeRatio'>,
               'GetPitchScale': <method 'GetPitchScale'>,
               'GetLatency': <method 'GetLatency'>,
               'SetTransientsOption': <method 'SetTransients

In [43]:
STRETCH_REF(41000, 1).__repr__()

'<RubberBand.RubberBandStretcher object at 0x000001A518C05D40>'

In [22]:
STRETCH_REF = RubberbandNet.lib.RubberBand.RubberBandStretcher
Rbs_Dict = RubberbandNet.lib.RubberBand.RubberBandStretcher.__dict__

In [20]:
help(RubberbandNet.lib.RubberBand.RubberBandStretcher)

Help on class RubberBandStretcher in module RubberBand:

class RubberBandStretcher(System.Object)
 |  Void .ctor(Int32, Int32, Options, Double, Double)
 |  
 |  Method resolution order:
 |      RubberBandStretcher
 |      System.Object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __str__(self, /)
 |      Return str(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from clr._internal.CLRMetatype
 |      Creat

In [5]:
opt = OptionManager()
o1 = RBTypes.Meta
o2 = RBTypes.Method
opt[o1]
class TOptSet(Enum):
    
    @classmethod
    def set_option_manager(cls, option_manager: OptionManager):
        cls._option_manager = option_manager

    def __getattr__(self, name):
        """
        Return the enum member matching `name`

        We use __getattr__ instead of descriptors or inserting into the enum
        class' __dict__ in order to support `name` and `value` being both
        properties for enum members (which live in the class' __dict__) and
        enum members themselves.
        """
        try:
            self._option_manager[self._member_map_[name]]
            return self._member_map_[name]
        except KeyError:
            raise AttributeError(name) from None

    def __getitem__(self, name):
        return self._member_map_[name]

class TOptSet2(TOptSet):
    option3 = RBTypes.Meta.value
    option4 = RBTypes.Method.value

TOptSet2.set_option_manager(opt)

NameError: name 'RBTypes' is not defined

In [49]:
TOptSet2._option_manager

In [46]:
TOptSet2.__dict__

mappingproxy({'_generate_next_value_': <function enum.Enum._generate_next_value_(name, start, count, last_values)>,
              '__module__': '__main__',
              '__doc__': 'An enumeration.',
              '_member_names_': ['option3', 'option4'],
              '_member_map_': {'option3': <TOptSet2.option3: <xpression.xtract.XT.xtract_configs.Option object at 0x00000238F1FFCFD0>>,
               'option4': <TOptSet2.option4: <xpression.xtract.XT.xtract_configs.Option object at 0x00000238F1FFCCA0>>},
              '_member_type_': object,
              '_value2member_map_': {<xpression.xtract.XT.xtract_configs.Option at 0x238f1ffcfd0>: <TOptSet2.option3: <xpression.xtract.XT.xtract_configs.Option object at 0x00000238F1FFCFD0>>,
               <xpression.xtract.XT.xtract_configs.Option at 0x238f1ffcca0>: <TOptSet2.option4: <xpression.xtract.XT.xtract_configs.Option object at 0x00000238F1FFCCA0>>},
              'option3': <TOptSet2.option3: <xpression.xtract.XT.xtract_configs.Opt

In [34]:
TOptSet.option1

<TOptSet.option1: <xpression.xtract.XT.xtract_configs.Option object at 0x00000238F1FFCA00>>

In [35]:
opt

[<RBTypes.Method: 'CLR.MethodBinding'>]

In [9]:
opt[RBTypes.Method]

[]

In [5]:
class RBTypes(str, Enum):
    Option = 'RubberBand.Options'
    Method = 'CLR.MethodBinding'
    Meta = 'clr._internal.CLRMetatype'

class CtypeParser:

    def __init__(self, map, module, ref=None, ctrlRef=None):
        self._module = module.__module__
        self.module = module
        self.map = map
        self.ref = ref if ref else self._module
        self.ctrlRef = ctrlRef if ctrlRef else self
        self.max_depth = 4
        self.current_depth = 0
        self._builtin_exception_max = 5
        self._builtin_exception_count = 0
        if hasattr(module, '__dir__') and type(map) == list: self.public_map = filter(lambda x: not x.startswith('_'), self.map)
        else: self.public_map = filter(lambda x: not x.startswith('_'), self.map.keys())
        self.cMap = {}
        self.dMap = {}

    def _init_class_map_(self, dMap_mode=False):
        self.ctrlRef.current_depth += 1
        if self.ctrlRef.current_depth >= self.ctrlRef.max_depth: return {}
        
        entry_template = {
            'cname': None,
            'pyname': None,
            'type': None,
            'value': None,
            'map': None
        }

        for i in self.public_map:
            value = getattr(self.module, i)
            csType = self._objecttype_format_(value)
            if csType == RBTypes.Method: rbType = RBTypes.Method.value
            elif csType == RBTypes.Option: rbType = RBTypes.Option.value
            elif csType == RBTypes.Meta: rbType = RBTypes.Meta.value
            elif 'builtins' in csType and self.module.__module__ == self.ref: rbType = csType
            else: 
                print(f'Unknown type: {csType}')
                if self._builtin_exception_count < self._builtin_exception_max: continue
                else: break
            
            cname = i
            pyname = str(cname).lower() if rbType == RBTypes.Method else cname
            if rbType == RBTypes.Option and hasattr(value, '__dir__') and callable(value.__dir__):
                imap = CtypeParser(value.__dir__(), value, self.ref, self.ctrlRef).dmap if dMap_mode else CtypeParser(value.__dir__(), value, self.ref, self.ctrlRef).cmap
                self.ctrlRef.current_depth -= 1
            else: imap = None

            self.cMap[i] = entry_template.copy()
            self.cMap[i]['cname'] = cname
            self.cMap[i]['pyname'] = pyname
            self.cMap[i]['type'] = rbType
            self.cMap[i]['value'] = value
            self.cMap[i]['map'] = imap
        return self.cMap if dMap_mode else self

    def _objecttype_format_(self, obj):
        mType = type(obj).__module__
        oType = type(obj).__name__
        return f'{mType}.{oType}'
    
    @property
    def dmap(self):
        return self._init_class_map_(True)
    
    @property
    def cmap(self):
        return self._init_class_map_()
    


In [ ]:
mtype = type(STRETCH_REF.CalculateStretch).__module__
otype = mtype + '.' + type(STRETCH_REF.CalculateStretch).__name__
otype

In [ ]:
parsed = CtypeParser(STRETCH_REF.__dict__, STRETCH_REF).dmap
iparsed = iter(parsed)

In [ ]:
RubberbandNet.lib.RubberBand.RubberBandStretcher.MemberwiseClone

In [ ]:
STRETCH_REF.PercussiveOptions.__dir__()
#hasattr(STRETCH_REF.PercussiveOptions, '__dir__')
type(FrozenSet)

In [ ]:
Rbs_Dict
STRETCH_REF.PercussiveOptions.__

In [ ]:
class RubberBandTypes:
    Options = STRETCH_REF.Options

In [ ]:
#cpp.lib.RubberBand.__dict__
#RubberBandEngine = create_model( 'RubberBandModel', **cpp.lib.RubberBand.RubberBandStretcher.__dict__)

model_dict = {}

def get_class_attr(obj, storage: dict = None):
    if storage is None:
        storage = {}
    for k, v in obj.__dict__.items():
        if str(type(v)).startswith("<method '") and str(type(v)).endswith("'>"):
            if not storage.get('methods'):
                storage['methods'] = []
            storage['methods'].append(k.replace('<method ', '').replace('>', ''))
        else:
            if not storage.get('attributes'):
                storage['attributes'] = {}
            if hasattr(v, '__dict__') or hasattr(v, '__slots__'):
                storage['attributes'][k] = {}
                get_class_attr(v, storage['attributes'][k])
            else:
                storage['attributes'][k] = v
    return storage
    


In [ ]:
from pprint import pprint
import datamodel_code_generator as dmcg
filetype = dmcg.InputFileType.Dict


rubber_attr = get_class_attr(STRETCH_REF, model_dict)
dmcg.generate(
    str(rubber_attr['attributes']), 
    input_file_type=filetype, 
    use_subclass_enum=True, 
    capitalise_enum_members=True, 
    class_name='RubberBandStretcher', 
    use_standard_collections=True,
    set_default_enum_member=True,
    apply_default_values_for_required_fields=True,
    original_field_name_delimiter='.',
    use_field_description=True,
    use_generic_container_types=True,
    use_one_literal_as_default=True
    )



In [ ]:
o = list(STRETCH_REF.DefaultOptions.__dir__())
f1 = filter(lambda x: not x.startswith('_'), o)
STRETCH_REF.Options
dotype = [type(getattr(STRETCH_REF.DefaultOptions, x)) for x in f1]
#[t == STRETCH_REF.Options for t in dotype]
#[i for i in RubberbandNet._assembly.DefinedTypes]
doa = [getattr(STRETCH_REF.DefaultOptions, x) for x in f1]
#[type(x) == callable for x in doa]
o


In [ ]:
Rbs_Dict

#### Load Target and Reference Audio Files

In [ ]:
target = WaveForm(r'xpression\xtract\Dogs_Normal.wav')
reference = WaveForm(r'xpression\xtract\Dogs_Angry.wav')
#tfm = sox.Transformer()


#### Adjust Target's Time Domain to Match the Reference. 

In [ ]:
flux_top_settings = pd.read_csv('top_three_settings_batch250_filters8.csv')
flux_top_settings = flux_top_settings.iloc[1].to_dict()
flux_settings = {k: v for k, v in flux_top_settings.items() if 'Unnamed' not in k and 'index' not in k}
target.reformers.flux.settings.HppsSettings = target.reformers.flux.settings.HppsSettings.construct(**flux_settings)

In [ ]:
tmod_target = target.reformers.flux.warp_to_match(reference)

In [ ]:
tmod_target._array = tmod_target._array.T
tmod_target.play

#### Extract Target & Reference Pitch

In [ ]:
pitch_info = tmod_target.informers.pitch # Create a pitch Informer object for both the target and reference
pitch_info2 = reference.informers.pitch
freq1 = pitch_info.frequency # Access the Frequency Feature of the Pitch Informers, with is returned as a pd.Series
freq2 = pitch_info2.frequency

In [ ]:
targetMP = tmod_target.informers.pitch
referenceP = reference.informers.pitch

TMDF = targetMP.frequency.to_frame()
TMDF['Time'] = targetMP.time
RDF = referenceP.frequency.to_frame()
RDF['Time'] = referenceP.time

#### Adjust the Reference Pitch Curve for the Target
Our goal here, is to take the reference curve (i.e. the curve we are trying to match), and adjust it so that it has the same median as the target. Though it is not neccessary here, since the two clips are spoken by the same person, in a real world setting, it would need to be adjusted to be in range of the target speakers natural pitch. 

Thus we create a comparison object, with the reference and target. Then align the two, to account for possible frame variance in the length of the reference, and time-domain adjust target. The rest is just frame formatting and such for the graphs, and not really neccessary.

In [ ]:
TmsRsComparison = targetMP.create_comparison(referenceP.frequency, targetMP.frequency)
RP, TP = TmsRsComparison.align
RPadjusted = pd.Series(np.where(RP > TmsRsComparison.means_diff, RP - 50, RP))

RADF = RPadjusted.to_frame(name='Frequency')
RADF['Time'] = referenceP.time
RADF = RADF[['Time', 'Frequency']]
RADF.Name = 'Reference-Adjusted'
TDF = TP.to_frame(name='Frequency')
TDF['Time'] = targetMP.time
TDF = TDF[['Time', 'Frequency']]
TDF.Name = 'Modification Target'


In [ ]:
# __ GRAPHING __
Rgx = WaveGraphX(RADF)
Tgx = WaveGraphX(TDF)
Rgx()
Rgx + Tgx
Rgx()

#### Create New Comparison, & Extract Curve Difference.
The next step is to calculate the difference between the two curves, now that they have the same median. This is needed so we know the warping factors to pass to
the pitch shifter, in order to achieve the same curve as the reference. 

Technically this portion is easier to calculate without using the `Comparison()` object, however, I decided to use it just for illustrative purposes. 

You can see via the `WaveGraphX()` below the code, that the calculated difference results in an almost perfect curve. 

In [ ]:
TRview = targetMP.create_comparison(RADF.Frequency, TDF.Frequency)
warping_vectors = TDF.Time.to_frame(name = 'Time')
warping_vectors['Frequency'] = TRview.diff.replace(np.nan, 0)
warping_vectors['Ratio'] = TRview.ratio.replace(0, np.nan).ffill()
warping_vectors['Frame'] = warping_vectors.Time * tmod_target.rate
# This is only here to make sure that we calculated our warping factors correctly.
# The actual value that we will be using to warp is the calculated RADF and TDF difference, listed above.
TDF_Warped = TDF.copy()
TDF_Warped.Frequency = TDF_Warped.Frequency + TRview.diff

In [ ]:
TWgx = WaveGraphX(TDF_Warped, name='Warped Target')
Rgx + TWgx
Rgx()

#### Use the Reformers.warp to Adjust the Target's Pitch Dynamically

The next step is pass our calculated warping vectors, to the warp `Reformer()`, and have it create a pitchmap or frequencymap for us. We can then access the warper's `engine` property to adjust anything we need to there. After that, we're ready to reform the audio! 

In [ ]:
warp = target.reformers.warp
warp.engine.frequency.value = 1.0
warp.frequency_map.add_to_map(warping_vectors.Frame.to_list(), warping_vectors.Ratio.to_list())
warp.engine.formant.Activation.activate
target_pitchModified = warp()

In [ ]:
Audio(shifted, rate=wf.rate)

In [ ]:
pm_audio = warper.map_pitch()

In [ ]:
shift = warper.shift_pitch()

In [ ]:
Audio(wf._array, rate=wf.rate)

In [ ]:
Audio(pm_audio, rate=wf.rate)

In [ ]:
def test(*args, **kwargs):
    if t1 := kwargs.pop('t1', False):
        print('Found t1')
        print(kwargs)
    else:
        print('No t1')

test(t1=1, t2=2, t3=3)

In [ ]:
rbs = RubberBandSettings()
rbs.use_R2_engine.formant.Activation.activate
rbs.engine.crisp.value = 4
rbs.engine.smoothing.Activation.activate
rbs.__compile__()

In [ ]:
source = SourceMap()
source.__map_to_source__(1.1, 2.6)
source.__map_to_source__(4.4, 4.2)
source.__map_to_source__(5.5, 6.6)
f = source.__source_to_file__()
print(open(f).read())
source.__del_file__()

In [ ]:
r2 = R2Engine()
f = r2.formant.dict()
s = SettingsManager()

In [ ]:
r2.__dict__

In [ ]:

r2.__getattribute__('formant').Activation.active

In [ ]:
s.storage

In [ ]:
def freq_ratio_to_cents(freq_ratio):
    return 0.0 if freq_ratio == 0 else 1200 * log2(freq_ratio)

In [ ]:
tdf = pd.read_csv('top_three_settings_batch250_filters8.csv')
tdf = tdf.drop(columns=['index', 'Unnamed: 0'])
tdfd = tdf.to_dict('index')
tdfl = deque()
for i in tdfd.values():
    s = list(i.values())
    s.pop(0)
    tdfl.append(s)

s = FormFluxSettings()
lp = LockPick(wf, s, wf2)
r = lp.manual_run(tdfl)
x = lp.xplore

In [ ]:
wf_input = x.data[1][0]
input_pitch, in_pitch_min, in_pitch_max = wf_input.reformers.pitch.pitch_net_caster()
in_pitch_freq = pd.Series(input_pitch.selected_array['frequency'], name='frequency')
in_pitch_strength = pd.Series(input_pitch.selected_array['strength'], name='strength')
pitch_time_start = []
pitch_time_end = []
for b in input_pitch.t_bins(): pitch_time_start.append(b[0]), pitch_time_end.append(b[1])

In [ ]:
reference_pitch, ref_pitch_min, ref_pitch_max = wf2.reformers.pitch.pitch_net_caster()
ref_pitch_freq = pd.Series(reference_pitch.selected_array['frequency'], name='frequency')
ref_pitch_strength = pd.Series(reference_pitch.selected_array['strength'], name='strength')
ref_pitch_time_start = []
ref_pitch_time_end = []
for b in reference_pitch.t_bins(): ref_pitch_time_start.append(b[0]), ref_pitch_time_end.append(b[1])
num_bends = len(ref_pitch_freq)

In [ ]:
ratios = ref_pitch_freq / in_pitch_freq
ratios = ratios.dropna()
cents = ratios.apply(freq_ratio_to_cents)
#HTML(cents.to_frame().to_html())

In [ ]:
pitch_bend = tfm.bend(num_bends,ref_pitch_time_start, ref_pitch_time_end, cents.to_list())

In [ ]:
tfm.clear_effects()

In [ ]:
test_start = [0.2]
test_end = [0.25]
test_cents = [1000]
pitch_bend = tfm.bend(1, test_start, test_end, test_end, frame_rate=80, oversample_rate=16)

In [ ]:
shifted_array = tfm.build_array(input_array = wf_input._array.T, sample_rate_in = wf_input.settings.rate)

In [ ]:
Audio(shifted_array, rate=wf_input.settings.rate)

In [ ]:
s = FormFluxSettings()
lp = LockPick(wf, s, wf2)
lp.filters = [
    'ola_win_size', 
    'ola_syn_hop_size', 
    'pv_zero_pad', 
    'hp_zero_pad', 
    'pv_syn_hop_size', 
    'hp_hop_size', 
    'hp_len_harm', 
    'hp_len_perc'
    ]

combinations = lp.start(batch_size=250)
batch = next(combinations)

In [ ]:
def xplore_saved_settings(df: pd.DataFrame, xplorer: Xplore):
    data = xplorer.data
    previous_settings = None
    for i in range(len(df)):
        settings = df.iloc[i]
        idx = settings['index']
        wf, s = data[idx]
        audio_array = wf._array.T
        rate = wf.settings.rate
        audio = Audio(audio_array, rate=rate)
        if previous_settings is not None:
            sdiff = previous_settings.compare(settings, result_names=('previous', 'current'))
        else:
            sdiff = settings
        previous_settings = settings
        yield display(audio, sdiff)

In [ ]:
def pitch_net_caster(waveform: WaveForm, runs = 4, i = 0, minimum = None, maximum = None, simplify = False):
    if minimum is None and maximum is None:
        p = waveform.forms.S.to_pitch_cc(very_accurate = True)
    else:
        p = waveform.forms.S.to_pitch_cc(pitch_floor = minimum, very_accurate=True, pitch_ceiling = maximum)
    smax = round(p.selected_array['frequency'].max())
    smin = round(p.selected_array['frequency'].min())
    i += 1
    if i != runs:
        smax = smax * 2
        smin = round(smin / 2)
        p, smin, smax = pitch_net_caster(waveform, runs, i, smin, smax)
    if i == runs and simplify == True:
        p = p.smooth()
    return p, smin, smax


In [ ]:
TOP_SETTINGS = []
def save_to_top_settings(df: pd.DataFrame, idx):
    s = df.loc[df['index'] == idx]
    TOP_SETTINGS.append(s)

In [ ]:
pnc, smin, smax = pitch_net_caster(w, simplify=True)

In [ ]:
p, pmin, pmax = w.reformers.pitch.pitch_net_caster()
pdf = pd.DataFrame(p.selected_array['frequency'])

In [ ]:
pdf.plot()

In [ ]:
wp = w.reformers.pitch.match_pitch(wf2)

In [ ]:
p1, wpmax, wpmin = w.reformers.pitch.pitch_net_caster()
p2, wf2max, wf2min = wf2.reformers.pitch.pitch_net_caster()

In [ ]:
p2.selected_array['frequency']

In [ ]:
s1 = pd.Series(p1.selected_array['frequency'])
s2 = pd.Series(p2.selected_array['frequency']).reindex(range(len(s1)), fill_value=1)
div = s2 / s1
div.replace(0, 1, inplace=True)
div = div.to_list()
window_len = p2.get_time_step()
window_size = int(window_len * wp.settings.rate)
#HTML(div.to_frame().to_html())


In [ ]:
w_shift = copy(w._array)

In [ ]:
len(w_shift.T.squeeze())

In [ ]:

#pitch_shifter = StftPitchShift(1024, 256, wp.settings.rate).shiftpitch(w_shift.T.squeeze(), div)
mpitch = pitcher.shiftpitch(w_shift.T, div, w.settings.rate)


In [ ]:
w._array

In [ ]:
wp.settings.rate

In [ ]:
HTML(pd.Series(w.audio.as_array().T.squeeze()).to_frame().to_html())

In [ ]:
Audio(mpitch, rate=48000)

In [ ]:
w1 = x.data[0][0]
w1 = w1.reformers.pitch.match_pitch(wf2)
w1 = w1.reformers.intensity.multiply_intensity(wf2)
w1.reformers.formant.settings.pitch_range_factor = 1.1
w1 = w1.reformers.formant.shift_formant(1.07)
w2 = x.data[1][0]
w2 = w2.reformers.pitch.match_pitch(wf2)
w2 = w2.reformers.intensity.multiply_intensity(wf2)
w2.reformers.formant.settings.pitch_range_factor = 1.1
w2 = w2.reformers.formant.shift_formant(1.07)
w3 = x.data[2][0]
w3 = w3.reformers.pitch.match_pitch(wf2)
w3 = w3.reformers.intensity.multiply_intensity(wf2)
w3.reformers.formant.settings.pitch_range_factor = 1.1
w3 = w3.reformers.formant.shift_formant(1.07)
a1 = Audio(w1.as_array(), rate=w1.sampling_frequency)
a2 = Audio(w2.as_array(), rate=w2.sampling_frequency)
a3 = Audio(w3.as_array(), rate=w3.sampling_frequency)
display(a1, a2, a3)

In [ ]:
w3_len = len(w3.values.T.squeeze())
w3_time  = w3_len / w3.sampling_frequency
w3_win_num = w3_time / w.reformers.pitch.settings.time_step
display(w3_len, w3_time, w3_win_num)

In [ ]:
wp1 = w1.reformers.pitch.match_pitch(wf2)
wi1 = wp1.reformers.intensity.multiply_intensity(wf2)
Audio(wi1._array, rate=wp1.settings.rate)

In [ ]:
Audio(wf2._array, rate=wf2.settings.rate)

In [ ]:
Audio(wp._array, rate=wp.settings.rate)

In [ ]:
wpsf = wp.reformers.formant.shift_formant(1.05)
Audio(wpsf.as_array(), rate = wpsf.sampling_frequency)

In [ ]:
Audio(wp._array, rate = wp.settings.rate)

In [ ]:
w.reformers.pitch.settings.pitch_floor = smin
w.reformers.pitch.settings.pitch_ceiling = smax
w.reformers.pitch.settings.octave_jump_cost = 0.5
wp = w.reformers.pitch.simplify.match_pitch(wf2)
#wp_audio_modulated = wp.audio.deepen_band_modulation()


In [ ]:
wp.reformers.intensity.settings.subtract_mean_pressure = False
wpi = wp.reformers.intensity.multiply_intensity(wf2)
Audio(wpi._array, rate = wpi.settings.rate)

In [ ]:
smax

In [ ]:
sa = pnc.selected_array['frequency']
df_sa = pd.DataFrame(sa)

In [ ]:
wmfcc = w.forms.S.to_mfcc(maximum_frequency = smax)
wmfccs = wmfcc.to_sound()

In [ ]:
wmfccs.as_array().shape

In [ ]:
wi = wp.reformers.intensity.multiply_intensity(wf2)

In [ ]:
Audio(wf2._array, rate=wf2.settings.rate)

In [ ]:
Audio(wi._array, rate=wp.settings.rate)

In [ ]:
pd.DataFrame(p1.selected_array['frequency']).plot()

In [ ]:

df_sa.plot()

In [ ]:
p1.selected_array['frequency'].max()

In [ ]:
p1.selected_array['frequency'].min()

In [ ]:
Audio(wp._array, rate=wp.settings.rate)

In [ ]:
Audio(wi._array, rate=wi.settings.rate)

In [ ]:
s

In [ ]:
top_xplorer = xplore_saved_settings(top_df, x)

In [ ]:
next(top_xplorer)

In [ ]:
x.current = 0

In [ ]:
a, s = x.get()
a

In [ ]:
a, s = x.get_next_set()
a

In [ ]:
a, s = x.good()
a

In [ ]:
a, s = x.bad()
a

In [ ]:
x.create_scoring_dataframe()

In [ ]:
x.scoring

In [ ]:
a, s = batch[1]
s

In [ ]:
a1, s1 = x.data[1]


In [ ]:
a1.reformers.flux.settings.HppsSettings = a1.reformers.flux.settings.HppsSettings.__class__(**s1)
a1.reformers.flux.settings.HppsSettings

In [ ]:
s1

In [ ]:
s

In [ ]:
a

In [ ]:
filters = ['ola_win_size', 'ola_syn_hop_size']
defaults = {'ola_win_size': 1024}
setting_combos = settings_tester(wf)
filtered_setting_combos = settings_filter(filters, setting_combos, defaults)
filtered_setting_combos

In [ ]:
# Test out the different parameter value combinations for wf.reformers.flux.settings.HppsSettings

def settings_tester(waveform: WaveForm):
    s = waveform.reformers.flux.settings.HppsSettings
    setting_combos = []
    for k, v in s.dict().items():
        if type(v) == bool:
            setting_combos.append(deque([True, False]))
        elif type(v) == int:
            if 'len' in k:
                setting_combos.append(deque([5, 10, 20]))
            elif 'size' in k:
                setting_combos.append(deque([1024, 2048, 4096]))
            elif 'pad' in k:
                setting_combos.append(deque([0, 10, 20]))
        elif type(v) == str:
            if 'type' in k:
                setting_combos.append(deque(['hann', 'sin']))
            elif 'mode' in k:
                setting_combos.append(deque(['binary', 'relative']))
    return setting_combos


In [ ]:
def settings_filter(filters, setting_combos, defaults = None):
    settings = {}
    settings_keys = [k for k in FormFluxSettings._FluxHppsSettings.__fields__.keys()]
    settings_defaults = [v for v in FormFluxSettings()._FluxHppsSettings().dict().values()]
    for i in range(len(settings_keys)): settings[settings_keys[i]] = i
    for f in filters: # for each filter in the list of filters
        if f in settings.keys(): # if the filter is a valid setting
            if defaults and f in defaults.keys(): # if the filter is in the defaults dictionary
                setting_combos[settings[f]] = deque([defaults[f]])
            else: setting_combos[settings[f]] = deque([settings_defaults[settings[f]]])
    return setting_combos


In [ ]:
def test_settings_combos(wf: WaveForm, reference_waveform: WaveForm, batch_size = 1, filter = None):
    wf.reformers.flux.use_hpps
    setting_combos = settings_tester(wf)
    if filter is not None: setting_combos = settings_filter(filter, setting_combos)
    setting_combos = itertools.product(*setting_combos)
    errors = 0
    total_runs = 0
    runs = []
    last_settings = None
    last_error = None
    last_changed = None
    

    for s in setting_combos:

        waveform = deepcopy(wf)
        k = [k for k in FormFluxSettings._FluxHppsSettings().__fields__.keys()]
        p = zip(k, s)
        current_setting = dict(p)

        if last_settings is None: last_settings = current_setting
        else:
            for i in range(len(last_settings)):

                if current_setting[k[i]] == last_settings[k[i]]:
                    continue
                else: 
                    last_changed = (k[i], current_setting[k[i]])
                    break
        
        total_runs += 1

        try:
            hpps_settings = FormFluxSettings()._FluxHppsSettings(**current_setting)
        except ValidationError:
            print(f'Validation Error on run {total_runs}: \n\n{dict(p)}')
            errors += 1
            time.sleep(0.5)
            continue

        waveform.reformers.flux.settings.HppsSettings = hpps_settings

        try:
            current_run = waveform.reformers.flux.warp_to_match(reference_waveform)
            runs.append((current_run, hpps_settings))
        except ValueError:
            errors += 1
            continue

        last_settings = current_setting
        
        clear_output(wait=True)
        display(f'Total Runs: {total_runs}')
        display(f'Errors: {errors}')
        display(f'Successful Runs: {len(runs)}')
        display(f'Last Changed: {last_changed}')
        display(f'Last Error: {last_error}')

        if len(runs) == batch_size:
            yield runs
            runs = []
    

In [ ]:
class Xplore:
    def __init__(self, audio_items: list):
        self.data = audio_items
        self.current = 0
        self.current_audio = None
        self.current_settings = None
        self.set_step = 18
        self.step = 1
        self.saved_settings = None
        self.setting_to_remove = None
        self.settings_type = FormFluxSettings()._FluxHppsSettings()
        self.scoring = self.create_scoring_dataframe()

    def compile_settings_dataframe(self):
        settings = []
        for a, s in self.data:
            settings.append(s.dict())
        df = pd.DataFrame(settings)
        return df

    def create_scoring_dataframe(self, data=None):
        catagories = list(self.settings_type.dict().keys())
        catagories = catagories.append('score')
        if data: df = pd.DataFrame(data, columns=catagories, index=range(len(data)))
        else: df = pd.DataFrame(columns=catagories)
        self.scoring = df
        return df
    
    def score(self):
        for c in self.scoring.columns:
            pass
            

    def items(self):
        for a, s in self.data:
            self.current_audio = a
            self.current_settings = s
            self.current += 1
            nparray = a._array.T
            sr = a.settings.rate
            yield Audio(nparray, rate=sr)

    def get(self):
        a, s = self.data[self.current]
        self.current += self.step
        audio = Audio(a._array.T, rate=a.settings.rate)
        self.current_audio = a
        self.current_settings = s
        return audio, s
    
    def get_next_set(self):
        self.current += self.set_step
        a, s = self.data[self.current]
        audio = Audio(a._array.T, rate=a.settings.rate)
        self.current_audio = a
        self.current_settings = s
        return audio, s
    
    def autoplay(self):
        a, s = self.data[self.current]
        self.current_audio = a
        self.current_settings = s
        self.current += 1
        nparray = a._array.T
        sr = a.settings.rate
        return Audio(nparray, rate=sr, autoplay=True), s
    
    def good(self, by_set=False):
        if self.current > 0: self.save_settings()
        scoring = self.current_settings.dict()
        scoring['score'] = 1
        if self.scoring.empty: self.create_scoring_dataframe(scoring)
        else: self.scoring.loc[len(self.scoring)] = scoring
        if by_set: return self.get_next_set()
        else: return self.get()
    
    def bad(self, by_set=False):
        if self.current > 0: self.to_remove()
        scoring = self.current_settings.dict()
        scoring['score'] = 0
        if self.scoring.empty: self.create_scoring_dataframe(scoring)
        else: self.scoring.loc[len(self.scoring)] = scoring
        if by_set: return self.get_next_set()
        else: return self.get()
        

    def to_remove(self):
        s = self.current_settings.dict()
        s['index'] = self.current
        if self.setting_to_remove is None:
            self.setting_to_remove = pd.DataFrame(s, index=[0])
        else:
            settings = pd.DataFrame(s, index=[0])
            self.setting_to_remove = pd.concat([self.setting_to_remove, settings], ignore_index=True)

    def save_settings(self):
        s = self.current_settings.dict()
        s['index'] = self.current
        if self.saved_settings is None:
            self.saved_settings = pd.DataFrame(s, index=[0])
        else:
            settings = pd.DataFrame(s, index=[0])
            self.saved_settings = pd.concat([self.saved_settings, settings], ignore_index=True)
        

In [ ]:
filters = ['ola_win_size', 'ola_syn_hop_size', 'pv_zero_pad', 'hp_zero_pad', 'pv_syn_hop_size', 'hp_hop_size', 'hp_len_harm', 'hp_len_perc']
batches = test_settings_combos(wf, wf2, batch_size=500, filter=filters)


In [ ]:
batch = next(batches)

In [ ]:
a1, s1 = batch[0]
a2, s2 = batch[1]
s1.dict() == s2.dict()

In [ ]:
x = Xplore(batch)
x.set_step = 8

In [ ]:
vcounts = [x.saved_settings[s].value_counts().nlargest(1) for s in x.saved_settings.columns if s != 'index']
data = [(v.values[0], v.index[0]) for v in vcounts]
columns = [v.name for v in vcounts]
#pd.DataFrame(vcounts)
data


In [ ]:
unique = [x.saved_settings[s].unique() for s in x.saved_settings.columns]
for s in x.saved_settings.columns:
    u = x.saved_settings[s].unique()
    c = []
    

In [ ]:
xdf = x.compile_settings_dataframe()

In [ ]:
x.saved_settings

In [ ]:
x.setting_to_remove

In [ ]:
type(x.scoring)

In [ ]:
a, s = x.get()
a

In [ ]:
a, s = x.get_next_set()
a

In [ ]:
a, s = x.good()
a

In [ ]:
a, s = x.bad(by_set=True)
a

In [ ]:
a, s = x.get_next_set()
a

In [ ]:
x.save_settings()

In [ ]:
x.saved_settings

In [ ]:
HTML(xdf.to_html())

In [ ]:
am, sm, = batch[717]

In [ ]:
Audio(am._array.T, rate=am.settings.rate)

### Phase Vocoder

In [ ]:
warped = wf.reformers.flux.use_phase_vox.warp_to_match(wf2)

In [ ]:
warped.forms.A

### Pitch Replacement

In [ ]:
warped.forms.reform_rate(44100)

In [ ]:
repitched = warped.reformers.pitch.match_pitch(wf2)

In [ ]:
Audio(repitched.audio.as_array(), rate=repitched.settings.rate)

### Intensity Replacement

In [ ]:
intensified = repitched.reformers.intensity.multiply_intensity(wf2)

In [ ]:
Audio(intensified.audio.as_array(), rate=intensified.settings.rate)